In [ ]:
import simpy
import numpy as np

class AcuteStrokeUnit:
    def __init__(self, env, run_length=365 * 5):
        self.env = env
        self.stroke_arrivals = 0
        self.tia_arrivals = 0
        self.complex_neurological_arrivals = 0
        self.other_arrivals = 0
        self.run_length = run_length
        self.stroke_rng = np.random.default_rng()
        self.tia_rng = np.random.default_rng()
        self.complex_neuro_rng = np.random.default_rng()
        self.other_rng = np.random.default_rng()

    def stroke_patient_generator(self):
        while True:
            interarrival_time = self.stroke_rng.exponential(scale=1.2)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrivals += 1
            print(f"Stroke patient {self.stroke_arrivals - 1} arrived at time {self.env.now}")
            post_asu_destination = self.stroke_rng.choice(['Rehab', 'ESD', 'Other'], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.stroke_arrivals - 1} left ASU for {post_asu_destination}")

    def tia_patient_generator(self):
        while True:
            interarrival_time = self.tia_rng.exponential(scale=9.3)
            yield self.env.timeout(interarrival_time)
            self.tia_arrivals += 1
            print(f"TIA patient {self.tia_arrivals - 1} arrived at time {self.env.now}")
            post_asu_destination = self.tia_rng.choice(['Rehab', 'ESD', 'Other'], p=[0.01, 0.01, 0.98])
            print(f"TIA patient {self.tia_arrivals - 1} left ASU for {post_asu_destination}")

    def complex_neurological_patient_generator(self):
        while True:
            interarrival_time = self.complex_neuro_rng.exponential(scale=3.6)
            yield self.env.timeout(interarrival_time)
            self.complex_neurological_arrivals += 1
            print(f"Complex Neurological patient {self.complex_neurological_arrivals - 1} arrived at time {self.env.now}")
            post_asu_destination = self.complex_neuro_rng.choice(['Rehab', 'ESD', 'Other'], p=[0.11, 0.05, 0.84])
            print(f"Complex Neurological patient {self.complex_neurological_arrivals - 1} left ASU for {post_asu_destination}")

    def other_patient_generator(self):
        while True:
            interarrival_time = self.other_rng.exponential(scale=3.2)
            yield self.env.timeout(interarrival_time)
            self.other_arrivals += 1
            print(f"Other patient {self.other_arrivals - 1} arrived at time {self.env.now}")
            post_asu_destination = self.other_rng.choice(['Rehab', 'ESD', 'Other'], p=[0.05, 0.10, 0.85])
            print(f"Other patient {self.other_arrivals - 1} left ASU for {post_asu_destination}")

    def run_simulation(self):
        stroke_generator = self.stroke_patient_generator()
        tia_generator = self.tia_patient_generator()
        complex_neuro_generator = self.complex_neurological_patient_generator()
        other_generator = self.other_patient_generator()

        self.env.process(stroke_generator)
        self.env.process(tia_generator)
        self.env.process(complex_neuro_generator)
        self.env.process(other_generator)

        yield self.env.timeout(self.run_length)

# Main simulation process
env = simpy.Environment()
acu = AcuteStrokeUnit(env)
env.process(acu.run_simulation())
env.run(until=acu.run_length)